In [1]:
import sys, os
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network

from brownie.network import connect, accounts, disconnect
from brownie_tokens import MintableForkToken

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)
connect('mainnet-fork')


Launching 'ganache-cli --port 8545 --gasLimit 12000000 --accounts 10 --hardfork istanbul --mnemonic brownie --fork https://eth-mainnet.alchemyapi.io/v2/AxU0de70ONfvbD-3_pQX0wUlaBK6g3G4 --chainId 1'...


In [3]:
pool_contract_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"  # cvxcrv pool addr
pool_contract = init_contract(contract_addr=pool_contract_addr)
pool_contract.name()

'Curve.fi Factory Plain Pool: cvxCRV'

In [4]:
pool_coin_addrs = []
for i in range(10):
    try:
        pool_coin_addrs.append(pool_contract.coins(i))
    except:
        break
pool_coin_addrs

['0xD533a949740bb3306d119CC777fa900bA034cd52',
 '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7']

In [5]:
pool_balances = pool_contract.get_balances()
total_coins = (pool_balances[1]+pool_balances[0])
print(f"coin 0 percentage: {pool_balances[0]/total_coins * 100}")
print(f"coin 1 percentage: {pool_balances[1]/total_coins * 100}")

coin 0 percentage: 41.58234538108825
coin 1 percentage: 58.41765461891174


add balance:

In [6]:
amount = int(pool_balances[0] / 15)  # swap size : 1/15th of cvxcrv pool balance
min_amount = int(0.99 * amount)  # 1% slippage
crv = MintableForkToken(pool_coin_addrs[0])  # crv is the first coin of cvxcrv-f pool
cvxcrv = MintableForkToken(pool_coin_addrs[1])

cvxcrv

<cvxCrvToken Contract '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7'>

In [7]:
cvxcrv._mint_for_testing(accounts[0], amount)

Transaction sent: 0x9d902bc0357ef936f328d06a17f31bb22ef9260010cd50e2ce9e9e6e307a80f1
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 1
  cvxCrvToken.transfer confirmed   Block: 13262178   Gas used: 51167 (0.43%)



In [8]:
cvxcrv.approve(pool_contract_addr, amount, {"from": accounts[0]})

Transaction sent: 0xfbd31f707c9218ee75e49df2b811efd7943a7db6913d676b8879b024c97c7828
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 2
  cvxCrvToken.approve confirmed   Block: 13262179   Gas used: 44106 (0.37%)



<Transaction '0xfbd31f707c9218ee75e49df2b811efd7943a7db6913d676b8879b024c97c7828'>

In [9]:
pool_contract.exchange(1, 0, amount, min_amount, {"from": accounts[0]})

Transaction sent: 0x0e66d900b895875f33d4f0431d147d74ec20ef379113e6f6e5b731302f3fe160
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 3
  Vyper_contract.exchange confirmed   Block: 13262180   Gas used: 78986 (0.66%)



<Transaction '0x0e66d900b895875f33d4f0431d147d74ec20ef379113e6f6e5b731302f3fe160'>

In [10]:
pool_balances = pool_contract.get_balances()
total_coins = (pool_balances[1]+pool_balances[0])
print(f"coin 0 percentage: {pool_balances[0]/total_coins * 100}")
print(f"coin 1 percentage: {pool_balances[1]/total_coins * 100}")

coin 0 percentage: 38.825468618137585
coin 1 percentage: 61.174531381862415


In [11]:
disconnect()

Terminating local RPC client...
